In [176]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711"   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")


CONNECTION SUCCESSFUL


In [177]:
#create a database
#mycursor = mydb.cursor()

#mycursor.execute("CREATE DATABASE project1")

In [178]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711", #REPLACE THIS WITH YOUR PASSWORD 
  database = "project1" #connecting to testDatabase
)

#redo table
mycursor = mydb.cursor()

In [212]:
#redo table
mycursor = mydb.cursor()

mycursor.execute("DROP TABLE IF EXISTS patients, physicians, pharmacies, drugs, prescriptions, adverse_interactions, alerts, pharmacy_fills, companies, contracts;")

In [214]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.physicians(
        ssn  varchar(11) NOT NULL,
        name varchar(128) NOT NULL,
        primary_specialty  varchar(256) NOT NULL,
        experience_years  decimal(5,2) NOT NULL,
        CHECK (experience_years >= 0),
        PRIMARY KEY (ssn)
    )
    '''
)

In [215]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.pharmacies(
        id  varchar(128) NOT NULL,
        name  varchar(256) NOT NULL,
        address varchar(256) NOT NULL,
        phone  varchar(20) NOT NULL,
        PRIMARY KEY (id)
    )
    '''
)

In [216]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.patients(
        ssn  varchar(11) NOT NULL,
        name  varchar(128) NOT NULL,
        address  varchar(256) NOT NULL,
        birth_date  date NOT NULL,
        physician_id  varchar(11) NOT NULL,
        PRIMARY KEY (ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians (ssn)
    )
    '''
)

In [217]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.drugs(
        id  varchar(20),
        name  varchar(256),
        PRIMARY KEY (name)
    )
    '''
)

In [218]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.prescriptions(
        id  varchar(20),
        patient_id  varchar(11),
        physician_id  varchar(11),
        drug_name  varchar(256),
        date  date,
        quantity  int,
        PRIMARY KEY (id),
        FOREIGN KEY (patient_id) REFERENCES patients (ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians (ssn),
        FOREIGN KEY (drug_name) REFERENCES drugs (name)
    )
    '''
)

In [219]:
mycursor.execute("CREATE INDEX idx_patient_drug ON prescriptions(patient_id, drug_name)")

In [220]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.adverse_interactions(
        drug_name  varchar(256),
        drug_name_2  varchar(256),
        PRIMARY KEY (drug_name, drug_name_2),
        FOREIGN KEY (drug_name) REFERENCES drugs (name),
        FOREIGN KEY (drug_name_2) REFERENCES drugs (name)
    )
    '''
)

In [221]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.pharmacy_fills(
        pharmacy_id  varchar(128),
        prescription_id  varchar(20),
        date date,
        cost  decimal(7,2),
        PRIMARY KEY (prescription_id, pharmacy_id),
        FOREIGN KEY (prescription_id) REFERENCES prescriptions (id),
        FOREIGN KEY (pharmacy_id) REFERENCES pharmacies (id)
    )
    '''
)

In [222]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.companies(
        id  varchar(20),
        name  varchar(128),
        address varchar(256),
        contact_phone  varchar(20),
        contact_name  varchar(128),
        PRIMARY KEY (id)
    )
    '''
)

In [223]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.contracts(
        id  varchar(20),
        drug_name  varchar(256),
        dosage  decimal(7,3),
        pharmacy_id  varchar(128),
        company_id  varchar(20),
        quantity  int,
        date  date,
        price  decimal(7,3),
        PRIMARY KEY (id),
        FOREIGN KEY (company_id) REFERENCES companies (id),
        FOREIGN KEY (pharmacy_id) REFERENCES pharmacies (id),
        FOREIGN KEY (drug_name) REFERENCES drugs (name)
    )
    '''
)

In [224]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1.alerts(
        patient_id  varchar(11),
        physician_id  varchar(11),
        alert_date  date,
        drug1  varchar(256),
        drug2  varchar(256),
        PRIMARY KEY (patient_id, physician_id, alert_date, drug1, drug2),
        FOREIGN KEY (patient_id) REFERENCES patients (ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians (ssn),
        FOREIGN KEY (patient_id, drug1) REFERENCES prescriptions (patient_id, drug_name),
        FOREIGN KEY (patient_id, drug2) REFERENCES prescriptions (patient_id, drug_name)
    )
    '''
)

In [225]:
#redo table
mycursor = mydb.cursor()

mycursor.execute("DROP TRIGGER IF EXISTS alerts;")

In [226]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TRIGGER alerts
    AFTER INSERT ON project1.prescriptions
    FOR EACH ROW
    BEGIN
        INSERT INTO project1.alerts (patient_id, physician_id, alert_date, drug1, drug2)
        SELECT DISTINCT P.patient_id, NEW.physician_id, NEW.date, P.drug_name, NEW.drug_name
        FROM project1.prescriptions AS P
        INNER JOIN project1.adverse_interactions A ON
            (A.drug_name = P.drug_name AND A.drug_name_2 = NEW.drug_name)
            OR (A.drug_name = NEW.drug_name AND A.drug_name_2 = P.drug_name)
        WHERE P.patient_id = NEW.patient_id;
    END;
    '''
)

In [227]:
import pandas as pd
import numpy as np

In [228]:
data = pd.read_csv("physicians.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO project1.physicians VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [229]:
data = pd.read_csv("patients.csv")
data['birthdate'] = pd.to_datetime(data['birthdate']).dt.date

for i,row in data.iterrows():
            sql = "INSERT INTO project1.patients VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [230]:
data = pd.read_csv("pharmacies.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO project1.pharmacies VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [231]:
data = pd.read_csv("drugs.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO project1.drugs VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [232]:
data = pd.read_csv("adverse_reactions.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO project1.adverse_interactions VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [233]:
data = pd.read_csv("prescriptions.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

for i,row in data.iterrows():
            sql = "INSERT INTO project1.prescriptions VALUES (%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [234]:
data = pd.read_csv("pharmacy_fills.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
data = data.replace({np.nan: None})

for i,row in data.iterrows():
            sql = "INSERT INTO project1.pharmacy_fills VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [235]:
data = pd.read_csv("companies.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO project1.companies VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [236]:
data = pd.read_csv("contracts.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

for i,row in data.iterrows():
            sql = "INSERT INTO project1.contracts VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


Write a trigger that inserts a tuple in the alerts table when a patient is prescribed a drug (i.e., after inserting a new row in the prescriptions table)  that adversely interacts with a drug the patient was earlier prescribed. So the event for the trigger is a patient being ascribed a drug (i.e., the insertion of a new row in the prescriptions table); the condition to be checked is whether the patient was earlier prescribed a drug that adversely interacts with the newly prescribed drug; the action is to insert a new tuple in the alerts table, where drug1 is the earlier prescribed drug and drug2 is the newly prescribed drug.

Find the physicians (ssn) who have most prescribed drugs which caused alerts (due to possible adverse interaction with a previously prescribed drug, not necessarily by the same physician).
Find the physicians (ssn) who have prescribed two drugs to the same patient which have adverse interactions.

Find the physicians who have prescribed most drugs supplied by company DRUGXO.

For each drug supplied by company PHARMASEE display the price (per unit of quantity) charged by that company for that drug along with the average price charged for that drug (by companies, not pharmacies). Note: As it happens in the data we supplied each drug is supplied by only one company, but your query should not be based on that.

For each drug and for each pharmacy, find the percentage of the markup (per unit of quantity) for that drug by that pharmacy.

For each drug, find the average time between when a patient was prescribed a drug and when the prescription was filled at a pharmacy. (You will need to extract the components of a date to compute this. mySQL provides functions for doing this and the official documentation here can help provide the appropriate functions: https://dev.mysql.com/doc/refman/8.4/en/date-and-time-functions.htmlLinks to an external site.).

For each pharmacy, find all the drugs that were prescribed to a patient and never filled at that pharmacy.

The hospital administration needs to retrieve from time to time primary specialty and years of experience of a given physician. Write a stored procedure that can be used to do that.

In [ ]:
mycursor.execute("DROP PROCEDURE IF EXISTS retrieve_phy")

mycursor.execute(
    '''
    CREATE PROCEDURE retrieve_phy (
        IN ssn_in VARCHAR(11),
        OUT out_primary_specialty VARCHAR(256),
        OUT out_experience_years DECIMAL(5, 2)
    )
    BEGIN
        SELECT primary_specialty, experience_years 
        INTO out_primary_specialty, out_experience_years
        FROM physicians
        WHERE ssn = ssn_in;
    END;
    '''
)

mycursor.execute("CALL retrieve_phy('118-66-5958', @primary_specialty, @experience_years)")
mycursor.execute("SELECT @primary_specialty, @experience_years")
result = mycursor.fetchall()
print(result)

[('Orthopedics', Decimal('3.00'))]
